# Why be subtle, let's classify with xgboost.

In [1]:
import lib.database_module as db
import lib.encoding_module as en
import lib.download_mine as down
import pandas as pd
import numpy as np
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.externals import joblib
import tqdm

In [2]:
vectors= db.fetch_all_category_vectors('local')

In [3]:
y = []
X = []

for v in vectors:
    v[1].insert(0,v[0])
    x = np.array(v[1])
    X.append(x)
vector_df = pd.DataFrame(X)
X_train, X_test, y_train, y_test = train_test_split(vector_df.iloc[:,1:],vector_df[0])

In [11]:
%%timeit -r1 -n1 

xgb = xgboost.XGBClassifier(silent = False, learning_rate=0.01, gamma=.2,
                            n_estimators = 1000,
                            nthread = 2,
                            max_depth= 5)
xgb.fit(X_train, y_train, verbose = True)

1 loop, best of 1: 39min 4s per loop


In [4]:
vectorizer = joblib.load('data/vectorizer.pkl')

Subcategories_dict = {'Sandwiches':'American sandwiches',
                'desserts':'cookies',
                'chemistry':'Industrial gases',
                'physics':'physicists',
                'sports cars': 'muscle cars',
                'psychology':'Popular psychology',
                'Arcade games':'Cancelled arcade games',
                'machine learning':'Classification algorithms',
                'cat breeds':'Natural cat breeds',
                'Earth sciences':'Geochemistry',
                'Breads':'Bread dishes',
                'Automotive technologies':'Vehicle dynamics',
                'belief':'Ignorance',
                'hygiene':'Ritual purification',
                'sports terminology':'martial arts terminology',
                'shoes':'Sneaker culture',
                'influenza':'Influenza researchers',
                'Children':'Sons of Odin',
                'physical quantities':'Density',
                'Physical exercise':'hydrotherapy',
                'health care':'medical robotics'
                }

Subcategory_number_dict = {}

for k,v in tqdm.tqdm_notebook(Subcategories_dict.items()):
    parent = down.wikipedia_get_page(k, category=True)
    child =  down.wikipedia_get_page(v, category=True)
    Subcategory_number_dict[v] ={'parent_name':k,
                                 'parent_id':int(parent.keys()[0]) , 
                                 'category_id': int(child.keys()[0]),
                                 'pages' :{p['pageid']: down.get_text(p['pageid']) 
                                           for p in down.wikipedia_get_pages_for_category(v) 
                                           if p['ns'] == 0}}
X_validation = []
y_validation = []
for k, v in Subcategory_number_dict.items():
    for key,text in v['pages'].items():
        y_validation.append(v['parent_id'])
        X_validation.append(en.clean_page_text(text))
X_validation = vectorizer.transform(X_validation)

In [ ]:
#gs_xgb = GridSearchCV(estimator, parameters, cv =3, n_jobs = -1, verbose =2)
#gs_xgb.fit(X_train,y_train)

print "Training Set Accuracy: {:.1%}".format(xgb.score(X_train,y_train))
print "Testing Set Accuracy: {:.1%}".format(xgb.score(X_test,y_test))

In [12]:
%%timeit
print "Validation Set Accuracy: {:.1%}".format(xgb.score(X_validation,y_validation))

Validation Set Accuracy: 66.8%
Validation Set Accuracy: 66.8%
Validation Set Accuracy: 66.8%
Validation Set Accuracy: 66.8%
1 loop, best of 3: 6.65 s per loop


In [5]:
y_validation[3:45]

[35880954,
 35880954,
 35880954,
 35880954,
 35880954,
 35880954,
 35880954,
 35880954,
 35880954,
 35880954,
 34955640,
 34955640,
 34955640,
 34955640,
 34955640,
 34955640,
 34955640,
 34955640,
 34955640,
 34955640,
 34955640,
 34955640,
 34955640,
 34955640,
 34955640,
 34955640,
 34955640,
 34955640,
 691015,
 691015,
 691015,
 691015,
 691015,
 691015,
 691015,
 691015,
 691015,
 691015,
 691015,
 691015,
 691015,
 691015]

In [ ]:
# make predictions for test data
y_pred = xgb.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

XGBoost Accuracy: Accuracy: 90.05% , but very slow.


## I'm sure that could be better. 
Let's try again with simpler clustering choices.  

In [ ]:
# Cosine similarity on the full category?

category_text = db.fetch_all_category_pages('local')

In [ ]:
train_text, test_text = train_test_split(category_text,test_size = .15)

In [ ]:
cate_train_dict = {}
for p in train_text:
    cate_train_dict[p[0]] = cate_train_dict.get(p[0],p[1])+ ' '+p[1]   

In [ ]:
cate_text_dict = {}
for k, text in cate_train_dict.items():
    cate_text_dict[k] = en.clean_page_text(text)

In [ ]:
categories, texts = zip(*cate_text_dict.items())
cat_vectorizer, cat_vectors = en.build_vectorizer(texts)

In [ ]:
def cos_similarity_compare(page_vector, cate_vectors, category_list):
    cos = cosine_similarity(cate_vectors,page_vector)
    best = cos.argmax()
    match1 = category_list[best]
    cos = np.delete(cos,best)
    category_list = np.delete(category_list,best)
    penultimate = cos.argmax()
    match2 = category_list[penultimate]
    return match1, match2

In [ ]:
x

In [ ]:
cate_test_list = []

for category, text in test_text:
    temp = en.clean_page_text(text)
    cate_test_list.append([category, text])

test_categories, test_texts = zip(*cate_test_list)
text_vectors = cat_vectorizer.transform(test_texts)
test_vectors = zip (test_categories, text_vectors)


test_results = []
for t in test_vectors:
    
    pred, pred2 = cos_similarity_compare(t[1].reshape(1,-1),
                                  cat_vectors,
                                  np.array(categories))
    actual = t[0]
    
#    print actual, pred, pred2
    if pred == actual:# or pred2 == actual: #Include 2 best matches
        test_results.append(int(1))
    else:
        test_results.append(int(0))
        
accuracy = float(sum(test_results))/len(cate_test_list)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
import sklearn.discriminant_analysis as da
import sklearn.cluster as cluster
import sklearn.neighbors as neighbors
import sklearn.neural_network as nn


In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors = 15, n_jobs =2)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
pred= knn.predict(X_test)
knn_results = zip(pred,y_test)
results = []
for k in knn_results:
    if k[0] == k[1]:
        results.append(int(1))
    else:
        results.append(int(0))
accuracy = float(sum(results))/len(results)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [6]:
import sklearn.neural_network as nn
mlpc = nn.MLPClassifier(verbose =1,
                        max_iter=2000, 
                        activation = 'logistic',
                        solver='adam',
                        hidden_layer_sizes=(100,) ,
                        tol=.00001)
                    
mlpc.fit(X_train, y_train)

Iteration 1, loss = 2.86719832
Iteration 2, loss = 2.54384491
Iteration 3, loss = 2.49737684
Iteration 4, loss = 2.49067569
Iteration 5, loss = 2.48681977
Iteration 6, loss = 2.48555888
Iteration 7, loss = 2.48390978
Iteration 8, loss = 2.48169172
Iteration 9, loss = 2.48056839
Iteration 10, loss = 2.47899730
Iteration 11, loss = 2.47755696
Iteration 12, loss = 2.47784834
Iteration 13, loss = 2.47419344
Iteration 14, loss = 2.47367019
Iteration 15, loss = 2.47342118
Iteration 16, loss = 2.47096548
Iteration 17, loss = 2.46940390
Iteration 18, loss = 2.46858743
Iteration 19, loss = 2.46755482
Iteration 20, loss = 2.46537427
Iteration 21, loss = 2.46477490
Iteration 22, loss = 2.46413340
Iteration 23, loss = 2.46134278
Iteration 24, loss = 2.45932580
Iteration 25, loss = 2.45837683
Iteration 26, loss = 2.45722546
Iteration 27, loss = 2.45527515
Iteration 28, loss = 2.45349523
Iteration 29, loss = 2.45154877
Iteration 30, loss = 2.45112865
Iteration 31, loss = 2.44796836
Iteration 32, los

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=1e-05, validation_fraction=0.1,
       verbose=1, warm_start=False)

In [7]:
print "Training Set Accuracy: {:.1%}".format(mlpc.score(X_train,y_train))
print "Testing Set Accuracy: {:.1%}".format(mlpc.score(X_test,y_test))
print "Validation Set Accuracy: {:.1%}".format(mlpc.score(X_validation,y_validation))

Training Set Accuracy: 97.7%
Testing Set Accuracy: 94.8%
Validation Set Accuracy: 70.8%


In [ ]:
da_class = da.LinearDiscriminantAnalysis()
da_class.fit_transform(X_train,y_train)

In [ ]:
da_class.predict(X_test)

In [ ]:
pred= da_class.predict(X_test)
pred_results = zip(pred,y_test)
results = []
for k in pred_results:
    if k[0] == k[1]:
        results.append(int(1))
    else:
        results.append(int(0))
accuracy = float(sum(results))/len(results)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
import sklearn.ensemble as ens

In [ ]:
tree = ens.RandomForestClassifier(n_estimators=30,n_jobs=2, verbose=2)
tree.fit(X_train,y_train)

In [ ]:
pred= tree.predict(X_test)
pred_results = zip(pred,y_test)
results = []
for k in pred_results:
    if k[0] == k[1]:
        results.append(int(1))
    else:
        results.append(int(0))
accuracy = float(sum(results))/len(results)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd = SGDClassifier(n_jobs=2, verbose=2, penalty='l1', )
sgd.fit(X_train,y_train)

In [ ]:
pred= sgd.predict(X_test)
pred_results = zip(pred,y_test)
results = []
for k in pred_results:
    if k[0] == k[1]:
        results.append(int(1))
    else:
        results.append(int(0))
accuracy = float(sum(results))/len(results)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def scoring_mlp(estimator, X_test, y_test):
    

In [ ]:
parameters = {'hidden_layer_sizes':((10,),(10,100),(100,10),(100,)),
              'solver':('sgd', 'adam'), 
              'activation':('logistic', 'tanh', 'relu')}
estimator = nn.MLPClassifier(verbose =1,
                        max_iter=2000,  
                        tol=.0001)

In [ ]:
gs = GridSearchCV(estimator, parameters, cv =2)

In [ ]:
gs.fit(X_train, y_train)

In [ ]:
gs.best_params_


## Mulitnomial naive bayes